# 📈 Notebook de Forecasting

Este notebook importa las librerías necesarias y carga el archivo de inferencia de ventas para el año 2025 en un DataFrame llamado `inferencia_df`.

In [1]:
# 🤖 Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

## Cargar archivo de inferencia en DataFrame
A continuación se carga el archivo `ventas_2025_inferencia.csv` en el DataFrame `inferencia_df`.

In [2]:
# Cargar el archivo de inferencia en un DataFrame
inferencia_df = pd.read_csv('../data/raw/inferencia/ventas_2025_inferencia.csv')
inferencia_df.head()

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [3]:
# Verificar estructura del archivo de competencia
comp_test = pd.read_csv('../data/raw/entrenamiento/competencia.csv')
print(comp_test.columns.tolist())
comp_test.head()

['fecha', 'producto_id', 'Amazon', 'Decathlon', 'Deporvillage']


,fecha,producto_id,Amazon,Decathlon,Deporvillage
0,2021-10-25,PROD_001,82.96,111.88,97.43
1,2021-10-25,PROD_002,112.56,108.61,115.58
2,2021-10-25,PROD_003,79.79,78.44,80.11
3,2021-10-25,PROD_004,72.60,67.29,74.45
4,2021-10-25,PROD_005,37.71,33.60,33.07


In [4]:
# Verificar si hay datos de competencia para 2025
comp_df = pd.read_csv('../data/raw/entrenamiento/competencia.csv')
comp_df['fecha'] = pd.to_datetime(comp_df['fecha'])
print("Años disponibles en competencia:", comp_df['fecha'].dt.year.unique())
print("Rango de fechas:", comp_df['fecha'].min(), "a", comp_df['fecha'].max())
comp_2025 = comp_df[comp_df['fecha'].dt.year == 2025]
print(f"\nRegistros de 2025: {len(comp_2025)}")
if len(comp_2025) > 0:
    comp_2025.head()
else:
    print("No hay datos de competencia para 2025")

Años disponibles en competencia: [2021 2022 2023 2024]
Rango de fechas: 2021-10-25 00:00:00 a 2024-11-30 00:00:00

Registros de 2025: 0
No hay datos de competencia para 2025


In [5]:
# 🤖 Transformación completa de inferencia_df siguiendo el proceso de entrenamiento

# 1. Convertir fecha a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Crear variables temporales
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['anio'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['es_fin_semana'] = (inferencia_df['dia_semana'].isin([5, 6])).astype(int)

# 3. Crear variable de festivos
festivos_espana = holidays.Spain(years=2025)
festivos = [str(fecha) for fecha in festivos_espana.keys()]
inferencia_df['es_festivo'] = inferencia_df['fecha'].dt.strftime('%Y-%m-%d').isin(festivos).astype(int)

# 4. Crear variables de eventos especiales (Black Friday y Cyber Monday 2025)
# Black Friday: 28 noviembre 2025
# Cyber Monday: 1 diciembre 2025
inferencia_df['es_Black_Friday'] = ((inferencia_df['mes'] == 11) & (inferencia_df['dia_mes'] == 28)).astype(int)
inferencia_df['es_Cyber_Monday'] = ((inferencia_df['mes'] == 12) & (inferencia_df['dia_mes'] == 1)).astype(int)

# 5. Crear variables temporales adicionales
inferencia_df['dia_anio'] = inferencia_df['fecha'].dt.dayofyear
inferencia_df['semana_anio'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['es_primer_dia_mes'] = (inferencia_df['dia_mes'] == 1).astype(int)
inferencia_df['es_ultimo_dia_mes'] = (inferencia_df['fecha'].dt.is_month_end).astype(int)
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['es_festivo_o_finsemana'] = ((inferencia_df['es_festivo'] == 1) | (inferencia_df['es_fin_semana'] == 1)).astype(int)

# 6. Crear variables de lag (usaremos valores vacíos ya que no tenemos histórico previo)
inferencia_df['lag_1'] = np.nan
inferencia_df['lag_2'] = np.nan
inferencia_df['lag_3'] = np.nan
inferencia_df['lag_4'] = np.nan
inferencia_df['lag_5'] = np.nan
inferencia_df['lag_6'] = np.nan
inferencia_df['lag_7'] = np.nan

# 7. Crear media móvil 7 días
inferencia_df['media_movil_7d'] = np.nan

# 8. Calcular porcentaje de descuento
inferencia_df['porcentaje_descuento'] = ((inferencia_df['precio_base'] - inferencia_df['precio_venta']) / inferencia_df['precio_base']) * 100

# 9. Cargar datos de competencia y calcular precio_competencia
# Como no hay datos de competencia para 2025, usaremos el promedio histórico por producto
competencia_df = pd.read_csv('../data/raw/entrenamiento/competencia.csv')
competencia_df['fecha'] = pd.to_datetime(competencia_df['fecha'])

# Calcular precio promedio histórico de competencia por producto
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
competencia_df['precio_competencia'] = competencia_df[competidores].mean(axis=1)

# Calcular el promedio histórico de precio_competencia por producto
precio_comp_promedio = competencia_df.groupby('producto_id')['precio_competencia'].mean().reset_index()
precio_comp_promedio.columns = ['producto_id', 'precio_competencia']

# Merge con inferencia_df
inferencia_df = inferencia_df.merge(
    precio_comp_promedio, 
    on='producto_id', 
    how='left'
)

# 10. Calcular ratio de precio
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# 11. Codificación One-Hot para variables categóricas
# Codificar 'nombre'
nombre_dummies = pd.get_dummies(inferencia_df['nombre'], prefix='nombre_h')
inferencia_df = pd.concat([inferencia_df, nombre_dummies], axis=1)

# Codificar 'categoria'
categoria_dummies = pd.get_dummies(inferencia_df['categoria'], prefix='categoria_h')
inferencia_df = pd.concat([inferencia_df, categoria_dummies], axis=1)

# Codificar 'subcategoria'
subcategoria_dummies = pd.get_dummies(inferencia_df['subcategoria'], prefix='subcategoria_h')
inferencia_df = pd.concat([inferencia_df, subcategoria_dummies], axis=1)

# 12. Verificar que tengamos todas las columnas necesarias
# Si faltan columnas dummy de entrenamiento, agregarlas con valor 0
columnas_entrenamiento = [
    'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23', 
    'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552', 
    'nombre_h_Columbia Silver Ridge', 'nombre_h_Decathlon Bandas Elásticas Set', 
    'nombre_h_Domyos BM900', 'nombre_h_Domyos Kit Mancuernas 20kg', 
    'nombre_h_Gaiam Premium Yoga Block', 'nombre_h_Liforme Yoga Pad', 
    'nombre_h_Lotuscrafts Yoga Bolster', 'nombre_h_Manduka PRO Yoga Mat', 
    'nombre_h_Merrell Moab 2 GTX', 'nombre_h_New Balance Fresh Foam X 1080v12', 
    'nombre_h_Nike Air Zoom Pegasus 40', 'nombre_h_Nike Dri-FIT Miler', 
    'nombre_h_Puma Velocity Nitro 2', 'nombre_h_Quechua MH500', 
    'nombre_h_Reebok Floatride Energy 5', 'nombre_h_Reebok Professional Deck', 
    'nombre_h_Salomon Speedcross 5 GTX', 'nombre_h_Sveltus Kettlebell 12kg', 
    'nombre_h_The North Face Borealis', 'nombre_h_Trek Marlin 7',
    'categoria_h_Fitness', 'categoria_h_Outdoor', 'categoria_h_Running', 'categoria_h_Wellness',
    'subcategoria_h_Banco Gimnasio', 'subcategoria_h_Bandas Elásticas', 
    'subcategoria_h_Bicicleta Montaña', 'subcategoria_h_Bloque Yoga', 
    'subcategoria_h_Cojín Yoga', 'subcategoria_h_Esterilla Fitness', 
    'subcategoria_h_Esterilla Yoga', 'subcategoria_h_Mancuernas Ajustables', 
    'subcategoria_h_Mochila Trekking', 'subcategoria_h_Pesa Rusa', 
    'subcategoria_h_Pesas Casa', 'subcategoria_h_Rodillera Yoga', 
    'subcategoria_h_Ropa Montaña', 'subcategoria_h_Ropa Running', 
    'subcategoria_h_Zapatillas Running', 'subcategoria_h_Zapatillas Trail'
]

for col in columnas_entrenamiento:
    if col not in inferencia_df.columns:
        inferencia_df[col] = 0

# 13. Eliminar registros de octubre y mantener solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 14. Reordenar columnas para que coincidan con el orden de entrenamiento
columnas_orden = [
    'fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria', 'precio_base', 
    'es_estrella', 'unidades_vendidas', 'precio_venta', 'ingresos', 'dia_semana', 
    'anio', 'mes', 'dia_mes', 'es_fin_semana', 'es_festivo', 'es_Black_Friday', 
    'es_Cyber_Monday', 'dia_anio', 'semana_anio', 'es_primer_dia_mes', 
    'es_ultimo_dia_mes', 'trimestre', 'es_festivo_o_finsemana', 'lag_1', 'lag_2', 
    'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'media_movil_7d', 
    'porcentaje_descuento', 'precio_competencia', 'ratio_precio'
] + columnas_entrenamiento

# Asegurar que todas las columnas existen
for col in columnas_orden:
    if col not in inferencia_df.columns:
        inferencia_df[col] = np.nan

inferencia_df = inferencia_df[columnas_orden]

# 15. Guardar el DataFrame transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)

print(f"✅ Transformación completada")
print(f"📊 Shape final: {inferencia_df.shape}")
print(f"📅 Registros de noviembre 2025: {len(inferencia_df)}")
print(f"📁 Guardado en: data/processed/inferencia_df_transformado.csv")

# Mostrar primeras filas
inferencia_df.head()

✅ Transformación completada
📊 Shape final: (720, 79)
📅 Registros de noviembre 2025: 720
📁 Guardado en: data/processed/inferencia_df_transformado.csv


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
169,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,NaN,135.00,NaN,...,False,False,False,False,False,False,False,False,True,False
170,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,NaN,86.39,NaN,...,False,False,False,False,False,False,False,False,True,False
171,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,NaN,74.09,NaN,...,False,False,False,False,False,False,False,False,True,False
172,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,NaN,34.76,NaN,...,False,False,False,False,False,False,False,True,False,False


In [6]:
# Verificar el resultado final
print("📊 RESUMEN DE TRANSFORMACIÓN")
print("="*60)
print(f"Total de registros: {len(inferencia_df)}")
print(f"Total de columnas: {len(inferencia_df.columns)}")
print(f"Período: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")
print(f"\n✅ Variables creadas correctamente")
print(f"✅ Solo registros de noviembre 2025")
print(f"✅ Archivo guardado en: data/processed/inferencia_df_transformado.csv")
print("\n📋 Primeras columnas:")
print(inferencia_df.columns.tolist()[:20])

📊 RESUMEN DE TRANSFORMACIÓN
Total de registros: 720
Total de columnas: 79
Período: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
Productos únicos: 24

✅ Variables creadas correctamente
✅ Solo registros de noviembre 2025
✅ Archivo guardado en: data/processed/inferencia_df_transformado.csv

📋 Primeras columnas:
['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria', 'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta', 'ingresos', 'dia_semana', 'anio', 'mes', 'dia_mes', 'es_fin_semana', 'es_festivo', 'es_Black_Friday', 'es_Cyber_Monday', 'dia_anio', 'semana_anio']


In [7]:
# Mostrar todos los nombres de columnas de inferencia_df sin omitir ninguna
pd.set_option('display.max_columns', None)
print(list(inferencia_df.columns))

['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria', 'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta', 'ingresos', 'dia_semana', 'anio', 'mes', 'dia_mes', 'es_fin_semana', 'es_festivo', 'es_Black_Friday', 'es_Cyber_Monday', 'dia_anio', 'semana_anio', 'es_primer_dia_mes', 'es_ultimo_dia_mes', 'trimestre', 'es_festivo_o_finsemana', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'media_movil_7d', 'porcentaje_descuento', 'precio_competencia', 'ratio_precio', 'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23', 'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552', 'nombre_h_Columbia Silver Ridge', 'nombre_h_Decathlon Bandas Elásticas Set', 'nombre_h_Domyos BM900', 'nombre_h_Domyos Kit Mancuernas 20kg', 'nombre_h_Gaiam Premium Yoga Block', 'nombre_h_Liforme Yoga Pad', 'nombre_h_Lotuscrafts Yoga Bolster', 'nombre_h_Manduka PRO Yoga Mat', 'nombre_h_Merrell Moab 2 GTX', 'nombre_h_New Balance Fresh Foam X 1080v12', 'nombr